In [5]:
#Optimizacion Bayesiana de hiperparametros de  rpart

#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,638366,34.1,1321371,70.6,1321371,70.6
Vcells,1134797,8.7,8388608,64.0,1682874,12.9


In [1]:
install.packages("rlist")

Installing package into 'C:/Users/ccarl/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)

also installing the dependency 'XML'




package 'XML' successfully unpacked and MD5 sums checked
package 'rlist' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\ccarl\AppData\Local\Temp\RtmpgvM8gi\downloaded_packages


In [2]:
require("data.table")
require("rlist")

Loading required package: data.table

Warning message:
"package 'data.table' was built under R version 4.2.3"
Loading required package: rlist

Warning message:
"package 'rlist' was built under R version 4.2.3"


In [3]:
require("rpart")
require("parallel")

Loading required package: rpart

Warning message:
"package 'rpart' was built under R version 4.2.3"
Loading required package: parallel



In [6]:
install.packages("mlrMBO")

Installing package into 'C:/Users/ccarl/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)

also installing the dependencies 'fastmatch', 'mlr', 'ParamHelpers', 'smoof', 'BBmisc', 'parallelMap'




package 'fastmatch' successfully unpacked and MD5 sums checked
package 'mlr' successfully unpacked and MD5 sums checked
package 'ParamHelpers' successfully unpacked and MD5 sums checked
package 'smoof' successfully unpacked and MD5 sums checked
package 'BBmisc' successfully unpacked and MD5 sums checked
package 'parallelMap' successfully unpacked and MD5 sums checked
package 'mlrMBO' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\ccarl\AppData\Local\Temp\RtmpgvM8gi\downloaded_packages


In [7]:

#paquetes necesarios para la Bayesian Optimization
require("DiceKriging")
require("mlrMBO")

Loading required package: DiceKriging

Warning message:
"package 'DiceKriging' was built under R version 4.2.3"
Loading required package: mlrMBO

Warning message:
"package 'mlrMBO' was built under R version 4.2.3"
Loading required package: mlr

Warning message:
"package 'mlr' was built under R version 4.2.3"
Loading required package: ParamHelpers

Warning message:
"package 'ParamHelpers' was built under R version 4.2.3"
Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} - please consider switching.

Loading required package: smoof

Warning message:
"package 'smoof' was built under R version 4.2.3"
Loading required package: checkmate


Attaching package: 'checkmate'


The following object is masked from 'package:DiceKriging':

    checkNames




In [8]:
#Defino la  Optimizacion Bayesiana

kBO_iter  <- 100   #cantidad de iteraciones de la Optimizacion Bayesiana

hs  <- makeParamSet(
          makeNumericParam("cp"       , lower= -1   , upper=    0.1),
          makeIntegerParam("minsplit" , lower=  1L  , upper= 8000L),  #la letra L al final significa ENTERO
          makeIntegerParam("minbucket", lower=  1L  , upper= 4000L),
          makeIntegerParam("maxdepth" , lower=  3L  , upper=   20L),
          forbidden = quote( minbucket > 0.5*minsplit ) )             # minbuket NO PUEDE ser mayor que la mitad de minsplit

ksemilla_azar  <- 604997   #cambiar por la primer semilla

In [9]:
#graba a un archivo los componentes de lista
#para el primer registro, escribe antes los titulos

loguear  <- function( reg, arch=NA, folder="./work/", ext=".txt", verbose=TRUE )
{
  archivo  <- arch
  if( is.na(arch) )  archivo  <- paste0( folder, substitute( reg), ext )

  if( !file.exists( archivo ) )  #Escribo los titulos
  {
    linea  <- paste0( "fecha\t", 
                      paste( list.names(reg), collapse="\t" ), "\n" )

    cat( linea, file=archivo )
  }

  linea  <- paste0( format(Sys.time(), "%Y%m%d %H%M%S"),  "\t",     #la fecha y hora
                    gsub( ", ", "\t", toString( reg ) ),  "\n" )

  cat( linea, file=archivo, append=TRUE )  #grabo al archivo

  if( verbose )  cat( linea )   #imprimo por pantalla
}

In [10]:
#particionar agrega una columna llamada fold a un dataset que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30 
# particionar( data=dataset, division=c(1,1,1,1,1), agrupa=clase_ternaria, seed=semilla)   divide el dataset en 5 particiones

particionar  <- function( data, division, agrupa="", campo="fold", start=1, seed=NA )
{
  if( !is.na( seed)  )   set.seed( seed )

  bloque  <- unlist( mapply(  function(x,y) { rep( y, x ) }, division, seq( from=start, length.out=length(division) )  ) )

  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))) )[1:.N],
           by= agrupa ]
}

In [11]:
#fold_test  tiene el numero de fold que voy a usar para testear, entreno en el resto de los folds
#param tiene los hiperparametros del arbol

ArbolSimple  <- function( fold_test, data, param )
{
  #genero el modelo
  modelo  <- rpart("clase_ternaria ~ .", 
                   data= data[ fold != fold_test, ],  #entreno en todo MENOS el fold_test que uso para testing
                   xval= 0,
                   control= param )

  #aplico el modelo a los datos de testing
  prediccion  <- predict( modelo, 
                          data[ fold==fold_test, ],  #aplico el modelo sobre los datos de testing
                          type= "prob")   #quiero que me devuelva probabilidades

  prob_baja2  <- prediccion[, "BAJA+2"]  #esta es la probabilidad de baja

  #calculo la ganancia
  ganancia_testing  <- data[ fold==fold_test ][ prob_baja2 > 1/40,  
                                                sum( ifelse( clase_ternaria=="BAJA+2", 117000, -3000 ) )] 

  return( ganancia_testing )  #esta es la ganancia sobre el fold de testing, NO esta normalizada
}

In [12]:
ArbolesCrossValidation  <- function( data, param, qfolds, pagrupa, semilla )
{
  divi  <- rep( 1, qfolds )  # generalmente  c(1, 1, 1, 1, 1 )  cinco unos

  particionar( data, divi, seed=semilla, agrupa=pagrupa )  #particiono en dataset en folds

  ganancias  <- mcmapply( ArbolSimple, 
                          seq(qfolds), # 1 2 3 4 5
                          MoreArgs= list( data, param), 
                          SIMPLIFY= FALSE,
                          mc.cores= 1 )   #se puede subir a qfolds si posee Linux o Mac OS

  data[ , fold := NULL ]

  #devuelvo la primer ganancia y el promedio
  ganancia_promedio  <- mean( unlist( ganancias ) )   #promedio las ganancias
  ganancia_promedio_normalizada  <- ganancia_promedio * qfolds  #aqui normalizo la ganancia
  
  return( ganancia_promedio_normalizada )
}

In [13]:
#esta funcion solo puede recibir los parametros que se estan optimizando
#el resto de los parametros, lamentablemente se pasan como variables globales

EstimarGanancia  <- function( x )
{
   GLOBAL_iteracion  <<-  GLOBAL_iteracion + 1

   xval_folds  <- 5
   ganancia  <- ArbolesCrossValidation( dataset,
                                        param= x, #los hiperparametros del arbol
                                        qfolds= xval_folds,  #la cantidad de folds
                                        pagrupa= "clase_ternaria",
                                        semilla= ksemilla_azar )

   #logueo 
   xx  <- x
   xx$xval_folds  <-  xval_folds
   xx$ganancia  <- ganancia
   xx$iteracion <- GLOBAL_iteracion
   loguear( xx,  arch= archivo_log )

   return( ganancia )
}

In [19]:
#Aqui empieza el programa

setwd("C:/Users/ccarl/Documents/labo2023v/src")   #Establezco el Working Directory

In [20]:
#cargo los datos
dataset  <- fread("./datasets/dataset_pequeno.csv")
#trabajo solo con los datos con clase, es decir 202107
dataset  <- dataset[ clase_ternaria!= "" ]

In [21]:
#creo la carpeta donde va el experimento
# HT  representa  Hiperparameter Tuning
dir.create( "./exp/",  showWarnings = FALSE ) 
dir.create( "./exp/HT3210/", showWarnings = FALSE )
setwd("./exp/HT3210/")   #Establezco el Working Directory DEL EXPERIMENTO

In [22]:
archivo_log  <- "HT321.txt"
archivo_BO   <- "HT321.RDATA"

In [23]:
#leo si ya existe el log, para retomar en caso que se se corte el programa
GLOBAL_iteracion  <- 0

if( file.exists(archivo_log) )
{
 tabla_log  <- fread( archivo_log )
 GLOBAL_iteracion  <- nrow( tabla_log )
}


In [24]:
#Aqui comienza la configuracion de la Bayesian Optimization

funcion_optimizar  <- EstimarGanancia

configureMlr( show.learner.output= FALSE)

In [25]:
#configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
#por favor, no desesperarse por lo complejo
obj.fun  <- makeSingleObjectiveFunction(
              fn=       funcion_optimizar,
              minimize= FALSE,   #estoy Maximizando la ganancia
              noisy=    TRUE,
              par.set=  hs,
              has.simple.signature = FALSE
             )

ctrl  <- makeMBOControl( save.on.disk.at.time= 600,  save.file.path= archivo_BO)
ctrl  <- setMBOControlTermination(ctrl, iters= kBO_iter )
ctrl  <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

surr.km  <- makeLearner("regr.km", predict.type= "se", covtype= "matern3_2", control= list(trace= TRUE))

In [26]:
#inicio la optimizacion bayesiana
if( !file.exists( archivo_BO ) ) {

  run  <- mbo( fun=     obj.fun, 
               learner= surr.km,
               control= ctrl)

} else  run  <- mboContinue( archivo_BO )   #retomo en caso que ya exista

Computing y column(s) for design. Not provided.



20230415 004332	-0.439524968134356	5635	664	9	5	51480000	1
20230415 004436	-0.70153931267123	2493	1066	20	5	50004000	2
20230415 004453	-0.21781695188547	2666	824	5	5	51315000	3
20230415 004517	-0.0973896665716892	6703	2278	9	5	50907000	4
20230415 004531	-0.951067709490599	7614	3282	4	5	49881000	5
20230415 004556	-0.38061960167106	7388	2521	14	5	50709000	6
20230415 004632	-0.890014183938911	4223	184	18	5	52065000	7
20230415 004645	0.0843459690135206	6259	399	16	5	0	8
20230415 004701	-0.517522284877487	4640	1832	5	5	50862000	9
20230415 004727	-0.108235307410359	7913	1450	18	5	50955000	10
20230415 004753	-0.678189504828437	4929	2412	13	5	50304000	11
20230415 004823	-0.545557937682606	5866	177	14	5	52524000	12
20230415 004848	-0.867436936092563	7120	848	11	5	50955000	13
20230415 004911	-0.973944059852511	7586	2990	20	5	51504000	14
20230415 004925	-0.183114870893769	3876	14	4	5	50520000	15
20230415 004948	-0.515891754883341	6927	3429	18	5	49749000	16


[mbo] 0: cp=-0.44; minsplit=5635; minbucket=664; maxdepth=9 : y = 5.15e+07 : 53.1 secs : initdesign

[mbo] 0: cp=-0.702; minsplit=2493; minbucket=1066; maxdepth=20 : y = 5e+07 : 63.9 secs : initdesign

[mbo] 0: cp=-0.218; minsplit=2666; minbucket=824; maxdepth=5 : y = 5.13e+07 : 17.0 secs : initdesign

[mbo] 0: cp=-0.0974; minsplit=6703; minbucket=2278; maxdepth=9 : y = 5.09e+07 : 24.0 secs : initdesign

[mbo] 0: cp=-0.951; minsplit=7614; minbucket=3282; maxdepth=4 : y = 4.99e+07 : 14.5 secs : initdesign

[mbo] 0: cp=-0.381; minsplit=7388; minbucket=2521; maxdepth=14 : y = 5.07e+07 : 24.8 secs : initdesign

[mbo] 0: cp=-0.89; minsplit=4223; minbucket=184; maxdepth=18 : y = 5.21e+07 : 35.4 secs : initdesign

[mbo] 0: cp=0.0843; minsplit=6259; minbucket=399; maxdepth=16 : y = 0 : 13.2 secs : initdesign

[mbo] 0: cp=-0.518; minsplit=4640; minbucket=1832; maxdepth=5 : y = 5.09e+07 : 16.6 secs : initdesign

[mbo] 0: cp=-0.108; minsplit=7913; minbucket=1450; maxdepth=18 : y = 5.1e+07 : 25.6 

20230415 005017	-0.906332591107852	3787	1807	14	5	50862000	17


[mbo] 1: cp=-0.906; minsplit=3787; minbucket=1807; maxdepth=14 : y = 5.09e+07 : 27.5 secs : infill_ei



...

[mbo] 98: cp=-0.789; minsplit=4143; minbucket=2062; maxdepth=20 : y = 5.06e+07 : 26.7 secs : infill_ei



20230415 013059	-0.403886347673691	7883	342	12	5	52056000	115


[mbo] 99: cp=-0.404; minsplit=7883; minbucket=342; maxdepth=12 : y = 5.21e+07 : 27.0 secs : infill_ei



20230415 013119	-0.204189457192221	7209	296	6	5	52056000	116


[mbo] 100: cp=-0.204; minsplit=7209; minbucket=296; maxdepth=6 : y = 5.21e+07 : 18.6 secs : infill_ei

Saved the final state in the file HT321.RDATA

